In [ ]:
import earthshine as etools 

import matplotlib.pylab as plt
import numpy as np
import scipy.integrate as integrate

from scipy.interpolate import interp1d

import time

################################################################################

# This will reload modules that have been edited
%load_ext autoreload
%autoreload 2


In [ ]:
time.time()

# Cross section

## Low energy

$$\nu n \rightarrow \mu^- p$$

https://arxiv.org/abs/1305.7513

Fig. 11

In [ ]:
energy_low = np.linspace(0.15,100,2000)
ypts0,x0,y0 = etools.neutrino_cross_section(energy_low)

#print(x)

plt.figure(figsize=(12,6))
#plt.subplot(1,2,1)
#plt.plot(energy,ypts);
plt.xlim(0.1,10000)
plt.xscale('log')

#plt.subplot(1,2,2)
plt.plot(x0,y0*1e38);
plt.xscale('log')
plt.ylabel("$\sigma [10^{-38} cm^2/nucleon]$", fontsize= 12)
plt.xlabel("$E_{\\nu} [GeV]$", fontsize= 12)
plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

## High energy

Measurement of the multi-TeV neutrino cross section with IceCube using Earth absorption


https://arxiv.org/abs/1711.08119

Fig. 1



In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pylab as plt


def tev_neutrino_cross_section(energy, curve='neutrino'):

    # Need this to read the proper file
    #data_path = pkg_resources.files('earthshine').joinpath('cross_section.csv')
    data_path = 'ice_cube_cross_section.csv'

    if curve=='weighted':
        indices = (0,1)    
    elif curve=='neutrino':
        indices = (2,3)
    elif curve=='antineutrino':
        indices = (4,5)
    elif curve=='result':
        indices = (6,7)

    data=np.loadtxt(data_path,dtype=str,delimiter=",",unpack=True, skiprows=2)
    x=10**(data[indices[0]][0:-2].astype(float))# GeV?  *1e9 # eV
    y=data[indices[1]][0:-2].astype(float)*1e-38

    #print(x)
    #print(y)
    
    # Convert     
    # Take out E dependence
    y *= x
    
    # Assume the cross section is flat
    #x = np.append(x,[10000])# Units are GeV
    #y = np.append(y,y[-1]) # Units are cm^2 / nucleon

    data_min_energy = min(x)
    data_max_energy = max(x)
    
    #print(data_min_energy, data_max_energy)

    if min(energy)<data_min_energy or max(energy)>data_max_energy:
        print("The energy range is outside of where the measurements took place!")
        print(f"measurements: {data_min_energy} - {data_max_energy}")
        print(f"energy:       {min(energy)} - {max(energy)}")
        exit()

    yfunction_xsec = interp1d(x,y)

    # Interpolated values
    ypts = yfunction_xsec(energy)

    # Return the interpolated xsec and the measurments (x,y)
    return ypts,x,y


In [ ]:
energy_high = np.linspace(350,10000,2000)
ypts1,x1,y1 = tev_neutrino_cross_section(energy_high, curve='neutrino')

#print(x)

plt.figure(figsize=(6,4))
plt.plot(x1,y1/x1,'o',label='Digitized points')
plt.plot(energy_high,ypts1/energy_high,lw=5,label='Interpolated data');
#plt.xlim(0.1,10000)
plt.ylabel(r"$\sigma_{\nu}/E_\nu \;$ [cm$^2$/nucleon]", fontsize= 18)
plt.xlabel(r"$E_{\nu}$ [GeV]", fontsize= 18)
plt.xscale('log')
plt.gca().tick_params(axis='both', which='major', labelsize=14)

plt.legend(fontsize=14)

plt.gca().set_facecolor('white')
plt.gcf().set_facecolor('white')

plt.tight_layout()


plt.figure(figsize=(8,8))

plt.subplot(2,1,1)
plt.plot(x1,y1,'o',label='Digitized points');
plt.plot(energy_high,ypts1,lw=5,label='Interpolation');
plt.xscale('log')
plt.ylabel(r"$\sigma_{\nu} \;$ [cm$^2$/nucleon]", fontsize= 18)
plt.xlabel(r"$E_{\nu}$ [GeV]", fontsize= 18)
plt.gca().tick_params(axis='both', which='major', labelsize=14)

plt.legend()

#plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

plt.gca().set_facecolor('white')
plt.gcf().set_facecolor('white')


plt.tight_layout()


plt.figure(figsize=(8,4))
plt.plot(x1,y1,'o',label='Digitized points');
plt.plot(energy_high,ypts1,lw=5,label='Interpolation');
plt.xscale('log')
plt.ylabel(r"$\sigma_{\nu} \;$  [cm$^2$/nucleon]", fontsize= 18)
plt.xlabel(r"$E_{\nu}$ [GeV]", fontsize= 18)
plt.gca().tick_params(axis='both', which='major', labelsize=14)

plt.ylim(1e-39,1e-34)

plt.legend()

#plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

plt.gca().set_facecolor('white')
plt.gcf().set_facecolor('white')


plt.tight_layout()

## PDG

https://pdg.lbl.gov/2021/reviews/rpp2021-rev-nu-cross-sections.pdf

Fig. 52.1

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pylab as plt



def pdg_neutrino_cross_section(energy, charge='neutrinos'):

    indices = [0,1,4,5]
    if charge == 'antineutrinos':
        indices = [2,3,6,7]
    
    # Need this to read the proper file
    #data_path = pkg_resources.files('earthshine').joinpath('cross_section.csv')
    data_path = 'pdg_cross_sections.csv'

    data=np.loadtxt(data_path,dtype=str,delimiter=",",unpack=True, skiprows=2)

    # Log scale
    x=data[indices[0]]
    y=data[indices[1]]
    
    mask = x!=''
    #x = x[mask]
    #y = y[mask]
    
    x=x[mask].astype(float)# GeV?  *1e9 # eV
    y=y[mask].astype(float)*1e-38

    #print(x)
    #print(y)
    
    xpts = x.tolist()
    ypts = y.tolist()

    # Lin scale
    x=data[indices[2]]
    y=data[indices[3]]
    
    mask = x!=''
    #x = x[mask]
    #y = y[mask]
    
    x=x[mask].astype(float)# GeV?  *1e9 # eV
    y=y[mask].astype(float)*1e-38

    #print(x)
    #print(y)
    
    x = np.array(xpts + x.tolist())
    y = np.array(ypts + y.tolist())

    
    # Assume the cross section is flat
    #x = np.append(x,[10000])# Units are GeV
    #y = np.append(y,y[-1]) # Units are cm^2 / nucleon

    data_min_energy = min(x)
    data_max_energy = max(x)
    
    print(data_min_energy, data_max_energy)

    if min(energy)<data_min_energy or max(energy)>data_max_energy:
        print("The energy range is outside of where the measurements took place!")
        print(f"measurements: {data_min_energy} - {data_max_energy}")
        print(f"energy:       {min(energy)} - {max(energy)}")
        exit()

    yfunction_xsec = interp1d(x,y)

    ypts = yfunction_xsec(energy)

    # Return the interpolated xsec and the measurments (x,y)
    return ypts,x,y


In [ ]:
energy_pdg = np.linspace(1,340,2000)
ypts2,x2,y2 = pdg_neutrino_cross_section(energy_pdg, charge='neutrinos')

#print(x)

plt.figure(figsize=(6,4))

#plt.subplot(1,2,2)
plt.plot(x2,y2,'o', label='Digitzed points');
plt.plot(energy_pdg, ypts2, lw=3, label='Interpolated data');

plt.xlim(0.1,1000)
plt.ylim(0.1e-38,1.5e-38)

plt.xscale('log')

plt.ylabel(r"$\sigma_{CC} / E_{\nu} [10^{-38} cm^2/nucleon]$", fontsize= 12)
plt.xlabel("$E_{\\nu} [GeV]$", fontsize= 12)
#plt.title("Interpolated Neutrino Cross Section", fontsize= 20)
plt.title("Derived from PDG Fig. 52.1", fontsize=14)

plt.legend(fontsize=14)

In [ ]:
plt.figure(figsize=(12,6))
#plt.subplot(1,2,1)
#plt.plot(energy,ypts);
#plt.xlim(0.1,10000)
plt.xscale('log')

#plt.subplot(1,2,2)
plt.plot(x2,y2,'o');
plt.plot(x1,y1/x1,'s')
plt.xscale('log')
plt.ylabel("$\sigma [10^{-38} cm^2/nucleon]$", fontsize= 12)
plt.xlabel("$E_{\\nu} [GeV]$", fontsize= 12)
plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

#plt.ylim(0.5e-38,0.8e-38)

In [ ]:
xtot = np.array(x2.tolist() + x1.tolist())
ytot = np.array(y2.tolist() + (y1/x1).tolist())

ytot *= xtot

plt.plot(xtot,ytot/xtot,'o')

plt.xscale('log')
plt.ylabel("$\sigma / E_{\nu} [10^{-38} cm^2/nucleon]$", fontsize= 12)
plt.xlabel("$E_{\\nu} [GeV]$", fontsize= 12)
plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

plt.legend(fontsize=14)

In [ ]:
elo = 1
ehi = 5000
#ehi = 200

x = np.linspace(elo,ehi,int(10*ehi))
#x = np.linspace(elo,ehi,int(ehi))

#x = np.linspace(elo,ehi,int(ehi))

#x = np.geomspace(elo,ehi,350)

energy = x
#print(x)

energy_width = energy[1] - energy[0]
print(f"Energy width: {energy_width} GeV")


yfunction = interp1d(xtot,ytot)
xsec = yfunction(energy)

plt.figure(figsize=(6,4))

plt.plot(xtot,ytot/xtot,'o', label='Digitized points')
plt.plot(energy,xsec/energy, lw=5, label='Interpolated data')

plt.xscale('log')
plt.ylabel(r"$\sigma / E_{\nu} [10^{-38} cm^2/nucleon]$", fontsize= 14)
plt.xlabel("$E_{\\nu} [GeV]$", fontsize= 18)
plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

plt.legend(fontsize=14)

#########################

plt.figure(figsize=(6,4))

plt.plot(energy,xsec, lw=5, label='Interpolated data')

plt.xscale('log')
plt.yscale('log')


plt.ylabel(r"$\sigma [10^{-38} cm^2/nucleon]$", fontsize= 14)
plt.xlabel("$E_{\\nu} [GeV]$", fontsize= 18)
plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

plt.legend(fontsize=14)


#########################

plt.figure(figsize=(6,4))

plt.plot(energy,xsec, lw=5, label='Interpolated data')

plt.xscale('log')


plt.ylabel(r"$\sigma [10^{-38} cm^2/nucleon]$", fontsize= 14)
plt.xlabel("$E_{\\nu} [GeV]$", fontsize= 18)
plt.title("Interpolated Neutrino Cross Section", fontsize= 20)

plt.legend(fontsize=14)

# Flux

Development of a General Analysis and Unfolding Scheme and its
Application to Measure the Energy Spectrum of Atmospheric Neutrinos
with IceCube

https://arxiv.org/pdf/1409.4535.pdf

Fig. 17

In [ ]:
################################################################################
#energy = np.arange(3,100000,10)


flux,a,b = etools.neutrino_flux(energy)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(energy,flux)
plt.xlabel('Energy')
plt.ylabel('Flux')
plt.xscale('log')
plt.yscale('log')



plt.subplot(1,2,2)
plt.plot(energy,flux*(energy**2))
plt.xlabel('Energy')
plt.ylabel('Flux')
plt.xscale('log')
plt.yscale('log')


# Different values for integration, depending on width
for width in [0.001, 0.01, 0.1, 0.5, 1, 2, 10]:
    xtmp = np.arange(1,5000,width)
    fluxtmp,a,b = etools.neutrino_flux(xtmp)
    integrated = integrate.trapz(fluxtmp, xtmp)
    print(f"width: {width:0.3f}   integrated: {integrated:.6f}")

In [ ]:
################################################################################
'''
energy = energytot


y_flux_new,a,b = etools.neutrino_flux(energy)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(energy,y_flux_new)
plt.xlabel('Energy')
plt.ylabel('Flux')



plt.subplot(1,2,2)
plt.plot(energy,y_flux_new*(energy**2))
plt.xlabel('Energy')
plt.ylabel('Flux')
plt.xscale('log')
plt.yscale('log')
'''

# Calculation

In [ ]:
# CMS dimensions
CMS_length = 21 # meters
CMS_width = 15 # meters
area_of_CMS = CMS_length*CMS_width 

#area_of_CMS /= 2 
# This is just because the cross section "looks" different from
# different angles, so this is just a guesstimate. 

length = 1.0 # length of rock "target" in meters?
density = 1.6e30 # Number of nucleons per m3?
#density_m3 = density_kg * ( 6e26) # Number of nucleons per kg

print(f"density: {density}")
print(f"length:  {length}")

In [ ]:
import pickle 

with open('cutoffs_file.pkl', 'rb') as fp:
    cutoffs = pickle.load(fp)
    print('cutoffs dictionary')
    #print(cutoffs)
    
print(len(cutoffs.keys()))

print(len(energy))


In [ ]:
k = list(cutoffs.keys())
v = list(cutoffs.values())

print(type(k))

for i in range(len(k)):
    print(k[i],v[i])

In [ ]:

#print()
#print(y_xsec_new)
#print()
#print(y_flux_new)
'''
cutoffs = {10e9: 25, \
           25e9: 55, \
           50e9: 100, \
           75e9: 140, \
           100e9: 180, \
           250e9: 420, \
           500e9: 760, \
           750e9: 1100, \
           1000e9: 1430, \
           2500e9: 3290, \
           5000e9: 6230, \
           7500e9: 9100, \
           10000e9: 11900
          }
'''

    

# Let's do the calculations for many chunks of rock
N_at_CMS = np.zeros(len(energy))

# Calculate the number of neutrinos coming out for a chunk of rock of length=length
# per steradian, at some given energy
N_per_chunk_of_rock = density*length*flux*xsec

# Multiply by the energy width to get the actual flux, not flux/GeV
N_per_chunk_of_rock *= energy_width

# These will be at different distances from CMS

start_time = time.time()

#plt.figure()

all_Ns = []
all_energies = []

energy_thresholds = list(cutoffs.keys())
distance_thresholds = list(cutoffs.values())
ncutoffs = len(energy_thresholds)

for distance in np.arange(1,200, length):
    
    #now = time.time()
    

    # Calculate the number of neutrinos coming out for 1 m^2 of rock of length=length
    # per steradian, at some given energy
    #N = density*length*flux*xsec

    # What is the solid angle at this distance?
    Omega = etools.solid_angle(CMS_length, CMS_width, distance)
    N = N_per_chunk_of_rock * Omega

    # How "many" of these 1m^2 chunks are there at distance=distance
    # *below* CMS (hemisphere)
    hemisphere_area = 2*np.pi*(distance**2) # in m^2
    N *= hemisphere_area

    if (int(distance)%10==0):
        now = time.time()
        diff = now - start_time
        print(f"diff: {diff:.3f} seconds     distance: {distance}   Omega: {Omega}     A_hemi: {hemisphere_area}")

    # Attenuation -- energy loss!
    #N_at_CMS_from_this_distance = np.zeros(len(energy))
    
    # Get energy_cutoff
    energy_cutoff = 0
    #for nc in range(ncutoffs-1,0,-1):
    #    #print(nc)
    #    if distance > distance_thresholds[nc]:
    #        energy_cutoff = (energy_thresholds[nc-1])*1e9
    #        #print(f"{energy_thresholds[nc]} {energy_cutoff}    distance: {distance} threshold {distance_thresholds[nc]}")
    #        break
    
    #print(int(distance))
    for i,e in enumerate(energy):
        #ke_final= etools.energy_loss_per_distance_traveled(e*1e9, distance, step_size=.1)
        
        if e>energy_cutoff:

            # Straight adaptive step size
            #ke_final = etools.final_energy_after_distance_traveled(e*1e9, distance, \
            #             mass=105e6, IS_E=True, ADAPTIVE_STEPSIZE=True)#step_size=1)

            # Assume distance is step size
            ke_final = etools.final_energy_after_distance_traveled(e*1e9, distance, \
                         mass=105e6, IS_E=True, step_size=distance)#, cutoffs=cutoffs)

            # Adaptive step size with cutoffs
            #ke_final = etools.final_energy_after_distance_traveled(e*1e9, distance, \
            #             mass=105e6, IS_E=True, ADAPTIVE_STEPSIZE=True, cutoffs=cutoffs)

            
            ke_final /= 1e9

            all_energies.append(ke_final)
            all_Ns.append(N[i])


            '''
            idx, central = etools.find_the_number(ke_final, energy)
            #print(e, ke_final, idx, central)
            if idx is not None:
                # N is the number coming out of the 1m^2 rock at distance=distance
                # N_at_CMS is the number of muons measured at CMS as different energies
                N_at_CMS_from_this_distance[idx] += N[i]
            '''
            #else:
              #print(e, ke_final, idx, central)
        else:
            #print(f"cutting off at {energy_threshold}")
            all_energies.append(0)
            all_Ns.append(N[i])
                
    #print(N_at_CMS_from_this_distance)
    #N_at_CMS += N_at_CMS_from_this_distance
    
    
    #if distance%10==0:
        #plt.plot(energy,N_at_CMS_from_this_distance,label=f'distance={distance} m')



In [ ]:
print(len(all_energies))
print(len(all_Ns))

In [ ]:
energies_at_CMS, N_at_CMS = etools.sum_up_shifted_energies(all_energies, all_Ns, \
                                                                energy_range=(0,ehi), nbins=int(10*ehi))

plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(energy, N_per_chunk_of_rock)
plt.xscale('log')
plt.xlabel("energy",fontsize=14)
plt.ylabel("number",fontsize=14)
#plt.title("energy vs number at CMS (per second)")
#plt.savefig("energy_vs_number_per_second")
##plt.yscale('log')
plt.legend()

plt.subplot(1,2,2)
plt.plot(energies_at_CMS, N_at_CMS,'o')
plt.xscale('log')
plt.xlabel("energy",fontsize=14)
plt.ylabel("number",fontsize=14)
#plt.title("energy vs number at CMS (per second)")
#plt.savefig("energy_vs_number_per_second")
plt.yscale('log')
plt.legend()


In [ ]:
energy

In [ ]:
# Now do for CMS 
# For grant???????????????????????????????????????????????????
#N_CMS = N*area_of_CMS
#plt.figure()
plt.figure(figsize=(10,8))
#plt.plot(energy, N)
plt.plot(energies_at_CMS, N_at_CMS,linewidth=4,label=r'$\nu$-induced muons')
plt.xscale('log')
plt.yscale('log')
plt.xlim(0,10000)
plt.xlabel('Energy of muons arriving at CMS (GeV)',fontsize=18)
plt.ylabel('Number of muons arriving at CMS (#/s)',fontsize=18)

plt.xticks(fontsize=18)


plt.legend(fontsize=24)
plt.savefig("background_earthshine.png")

In [ ]:
energies_at_CMS[0:100]

In [ ]:
# Now do for CMS 
#N_CMS = N*area_of_CMS
N_CMS_month = N_at_CMS * 3e7 /  12
plt.figure()
#plt.plot(energy, N)
plt.plot(energies_at_CMS, N_at_CMS,label='# muons at CMS/s')
plt.plot(energies_at_CMS, N_CMS_month,label='# muons at CMS/month')
plt.xscale('log')
plt.yscale('log')

plt.legend()


idxlo=50
integrated = integrate.trapz(N_CMS_month[idxlo:], energies_at_CMS[idxlo:])
print(f"The integrated number is {integrated} per month for an energy range between {energies_at_CMS[idxlo]} and {energy[-1]} GeV")

integrated = integrate.trapz(N_CMS_month[0:idxlo], energies_at_CMS[0:idxlo])
print(f"The integrated number is {integrated} per month for an energy range between {energies_at_CMS[0]} and {energy[idxlo]} GeV")

integrated = integrate.trapz(N_CMS_month[1:idxlo], energies_at_CMS[1:idxlo])
print(f"The integrated number is {integrated} per month for an energy range between {energies_at_CMS[1]} and {energy[idxlo]} GeV")

integrated = integrate.trapz(N_CMS_month[0:2], energies_at_CMS[0:2])
print(f"The integrated number is {integrated} per month for an energy range between {energies_at_CMS[0]} and {energy[2]} GeV")


plt.title("Number at CMS per month")
plt.savefig("integrated_number per month.png")

################################################################################


#ASK BELLIS!! I THINK I DID SOMETHING WRONG HERE 
#Putting in vals for en restricts the energy range, not the radius, and we want to restrict the radius. So how do I fix that? 

#plt.show()
en=np.arange(1,200)
print(en)
enlist=en.tolist()
'''
#print(enlist)
xsec_for_mult_plot=yfunction_xsec[enlist]
flux_for_mult_plot=yfunction_flux[enlist]
mult_plot= xsec_for_mult_plot*flux_for_mult_plot
plt.plot(mult_plot,enlist)
plt.show()
'''

flux_new=etools.neutrino_flux(enlist)
#fluxy=ypts not needed right now
xsec_new=etools.neutrino_cross_section(enlist)
plt.figure() 
#print(ypts)
#print(flux_new[0])
tot= flux_new[0]*xsec_new[0]#*density*en[-1]
#print(tot)
plt.plot(en, tot)
#plt.xscale("log")
plt.yscale("log")
plt.xlabel("energy",fontsize=14)
plt.ylabel("flux*cross section",fontsize=14)
#plt.show()

#print(en[-1])


